In [3]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import seaborn as sns
import csv
from scipy.io import arff
from sklearn import datasets
from sklearn.cluster import KMeans
import numpy as np
from kmodes.kmodes import KModes
from kmodes.util.dissim import ng_dissim


In [7]:
#data = arff.loadarff('universityPolicyTrainingData.arff')
permit = pd.read_csv('healthCareRulesPermitted.csv')
denied = pd.read_csv('healthCareRulesDenied.csv')
df = pd.DataFrame(permit)
#print(denied)
#refine = pd.read_csv('step1ExtractedRulesfalseNegatives.csv')
#df = pd.DataFrame(refine)
act = pd.concat([permit,denied])
#print(data)

df= df.drop(columns ='Unnamed: 13')

print()
print(df.describe())
df.head()


       action position    type    oward    uward     uteam otreatingteam  \
count    9000     9000    9000     9000     9000      9000          9000   
unique      3        2       2        2        2         4             4   
top      read    nurse  hritem  oncward  oncward  oncteam1      carteam2   
freq     5000     5509    5000     4507     4605      2311          2293   

       opatient    oauthor topic specialty uagentfor     user  
count      9000       9000  9000      9000      9000     9000  
unique        4          6     4         4         4       10  
top     carpat2  oncnurse1  note  oncology   carpat1  carpat1  
freq       2294       1547  3425      2350      2305     1145  


,action,position,type,oward,uward,uteam,otreatingteam,opatient,oauthor,topic,specialty,uagentfor,user
0,read,nurse,hritem,carward,carward,oncteam2,oncteam1,carpat1,carnurse1,oncology,oncology,carpat1,carnurse1
1,additem,doctor,hr,carward,oncward,carteam2,carteam2,carpat1,oncnurse1,note,note,oncpat1,oncdoc2
2,addnote,doctor,hr,carward,carward,oncteam1,carteam2,carpat2,oncdoc1,cardiology,nursing,carpat2,carpat2
3,additem,nurse,hr,carward,carward,carteam2,oncteam1,carpat2,oncnurse1,note,oncology,carpat1,oncnurse1
4,read,doctor,hritem,oncward,carward,carteam2,carteam2,oncpat2,oncdoc1,note,note,oncpat1,oncnurse2


In [8]:
#print(df.describe())

df.isnull().values.any()
df["position"] = df["position"].astype('category')
df["oward"] = df["oward"].astype('category')
df["opatient"] = df["opatient"].astype('category')
df["oauthor"] = df["oauthor"].astype('category')
df["otreatingteam"] = df["otreatingteam"].astype('category')
df["type"] = df["type"].astype('category')
df["uteam"] = df["uteam"].astype('category')
df["uward"] = df["uward"].astype('category')
df["action"] = df["action"].astype('category')
df["topic"] = df["topic"].astype('category')
df["specialty"] = df["specialty"].astype('category')
df["uagentfor"] = df["uagentfor"].astype('category')
df["user"] = df["user"].astype('category')


act.isnull().values.any()
act["position"] = act["position"].astype('category')
act["oward"] = act["oward"].astype('category')
act["opatient"] = act["opatient"].astype('category')
act["oauthor"] = act["oauthor"].astype('category')
act["otreatingteam"] = act["otreatingteam"].astype('category')
act["type"] = act["type"].astype('category')
act["uteam"] = act["uteam"].astype('category')
act["uward"] = act["uward"].astype('category')
act["action"] = act["action"].astype('category')
act["topic"] = act["topic"].astype('category')
act["specialty"] = act["specialty"].astype('category')
act["uagentfor"] = act["uagentfor"].astype('category')
act["user"] = act["user"].astype('category')

df.head(30)
print(df.dtypes)

action           category
position         category
type             category
oward            category
uward            category
uteam            category
otreatingteam    category
opatient         category
oauthor          category
topic            category
specialty        category
uagentfor        category
user             category
dtype: object


In [9]:
# Attributes Frequencies
datafreq = pd.DataFrame()
for col in df.columns:
    datafreq=df[col].value_counts()
    #print (datafreq)
    
#print(datafreq)



In [10]:
# Correlation Function

def cramers_v(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher,
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

# Writing Data Correlation in a CSV
with open("dcorr.csv",'w',newline='') as f:
    for col in df.columns:
    
        writer = csv.writer(f, dialect='excel')
        for col2 in df.columns:
            
            confusion_matrix = pd.crosstab(df[col], df[col2]).as_matrix()
            temp = cramers_v(confusion_matrix)
            writer.writerow({temp, col2 , col })
    f.close()



/Users/aldairi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [41]:
data = df
k=14
datasize= len(data.index)
print(datasize)
km = KModes(n_clusters=k, init='Cao', n_init=1, verbose=2)
df_dummy = pd.get_dummies(df)

clusters = km.fit_predict(data)
df_dummy['clusters'] = clusters

# Print the cluster centroids

centroids=km.cluster_centroids_

print(centroids)

9000
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 97, cost: 49564.0
Run 1, iteration: 2/100, moves: 0, cost: 49564.0
[['read' 'nurse' 'hritem' 'carward' 'oncward' 'oncteam1' 'oncteam1'
  'oncpat2' 'oncnurse1' 'note' 'oncology' 'carpat1' 'oncpat2']
 ['addnote' 'doctor' 'hr' 'oncward' 'carward' 'carteam1' 'carteam2'
  'carpat1' 'oncdoc2' 'nursing' 'note' 'oncpat1' 'carpat1']
 ['read' 'nurse' 'hritem' 'oncward' 'carward' 'oncteam2' 'oncteam2'
  'oncpat1' 'oncdoc1' 'nursing' 'nursing' 'carpat2' 'oncnurse1']
 ['read' 'doctor' 'hritem' 'oncward' 'oncward' 'carteam2' 'carteam2'
  'carpat2' 'carnurse1' 'cardiology' 'cardiology' 'oncpat2' 'oncpat1']
 ['additem' 'nurse' 'hr' 'carward' 'carward' 'oncteam1' 'carteam1'
  'carpat2' 'oncnurse2' 'cardiology' 'note' 'carpat2' 'oncpat1']
 ['additem' 'nurse' 'hr' 'oncward' 'oncward' 'oncteam2' 'oncteam2'
  'carpat2' 'oncdoc2' 'note' 'cardiology' 'oncpat1' 'oncpat2']
 ['read' 'doctor' 'hri

In [42]:
cluster_map = pd.DataFrame()
cluster_map['data_index'] = data.index.values
cluster_map['cluster'] = km.labels_

#valuescount=pd.DataFrame()
#temp=[]

for i in range(0, 9):
    ind=cluster_map.cluster == i
    #print(ind)

In [43]:
cluster_map
ndf=pd.concat([data,cluster_map], axis=1)
ndf=ndf.drop(columns='data_index')
ndf

,action,position,type,oward,uward,uteam,otreatingteam,opatient,oauthor,topic,specialty,uagentfor,user,cluster
0,read,nurse,hritem,carward,carward,oncteam2,oncteam1,carpat1,carnurse1,oncology,oncology,carpat1,carnurse1,0
1,additem,doctor,hr,carward,oncward,carteam2,carteam2,carpat1,oncnurse1,note,note,oncpat1,oncdoc2,1
2,addnote,doctor,hr,carward,carward,oncteam1,carteam2,carpat2,oncdoc1,cardiology,nursing,carpat2,carpat2,11
3,additem,nurse,hr,carward,carward,carteam2,oncteam1,carpat2,oncnurse1,note,oncology,carpat1,oncnurse1,0
4,read,doctor,hritem,oncward,carward,carteam2,carteam2,oncpat2,oncdoc1,note,note,oncpat1,oncnurse2,8
5,additem,nurse,hr,oncward,oncward,carteam2,carteam1,oncpat2,oncnurse1,nursing,cardiology,carpat1,carpat1,5
6,addnote,doctor,hr,oncward,carward,oncteam1,oncteam1,carpat1,oncdoc1,cardiology,note,oncpat2,carpat1,1
7,additem,nurse,hr,oncward,oncward,carteam2,carteam2,carpat1,carnurse1,oncology,note,carpat1,carnurse1,1
8,additem,nurse,hr,carward,carward,carteam1,carteam1,oncpat1,oncnurse1,note,note,oncpat1,oncdoc1,4
9,read,doctor,hritem,oncward,oncward,oncteam1,oncteam1,oncpat2,carnurse1,note,note,carpat2,carpat2,10


In [44]:
#centroids
ndf.groupby('cluster').count()

,action,position,type,oward,uward,uteam,otreatingteam,opatient,oauthor,topic,specialty,uagentfor,user
cluster,,,,,,,,,,,,,
0,1349,1349,1349,1349,1349,1349,1349,1349,1349,1349,1349,1349,1349
1,701,701,701,701,701,701,701,701,701,701,701,701,701
2,1003,1003,1003,1003,1003,1003,1003,1003,1003,1003,1003,1003,1003
3,653,653,653,653,653,653,653,653,653,653,653,653,653
4,732,732,732,732,732,732,732,732,732,732,732,732,732
5,731,731,731,731,731,731,731,731,731,731,731,731,731
6,492,492,492,492,492,492,492,492,492,492,492,492,492
7,708,708,708,708,708,708,708,708,708,708,708,708,708
8,639,639,639,639,639,639,639,639,639,639,639,639,639


In [47]:
# Calculating Attributes Effectevness in each cluster
temp = pd.DataFrame()
datafreq = pd.DataFrame()
extract = pd.DataFrame()
rules = pd.DataFrame(columns=['action' ,'position',
'type'             ,
'oward'            ,
'uward'            ,
'uteam'            ,
'otreatingteam'    ,
'opatient'         ,
'oauthor'          ,
'topic'            ,
'specialty'        ,
'uagentfor'        ,
'user' ])
print(datafreq)
clusTemp = pd.DataFrame()
for i in range(k):
    print()
    print("CLUSTER NUMBER", i)
    clusTemp = ndf.loc[ndf['cluster'] == i]
    clussize= len(clusTemp.index)
    #print(clussize)
    for col in df.columns:
        datafreq = act[col].value_counts().to_dict()
        #datafreq= pd.DataFrame.from_dict(datafreq, )
        temp=clusTemp[col].value_counts().to_dict()
        #temp= pd.DataFrame.from_dict(temp)
        #print(temp)
        #print(datafreq)
        for key, value in datafreq.items():
            #if key not in temp.keys():
                #print (key, 'is not in the cluster')
        #    datafreq["count"] = df.loc[col == attr].size()
            #print(key)
            x = value/datasize
            #print(x)
            for key1, value1 in temp.items():
                
                if key == key1:
                    y = value1/clussize
                    #print(y)
                    
                    if y-x >= 0.020:
                        #print('the effectevniess of ', key, y, x, 'is: ', y-x)
                        #l.append(key)
                        #print(col)
                        print(col,'=',key,',',end="")
                    #if y-x < -0.1:
                         #print (col,'!=',key,',',end="")
                        #print(key,'is not effectevniess of ',  y, x, 'is: ', y-x)

  


Empty DataFrame
Columns: []
Index: []

CLUSTER NUMBER 0
user = oncpat2 ,
CLUSTER NUMBER 1
action = addnote ,type = hr ,user = carpat1 ,
CLUSTER NUMBER 2
uteam = oncteam2 ,otreatingteam = oncteam2 ,topic = nursing ,
CLUSTER NUMBER 3
uteam = carteam2 ,otreatingteam = carteam2 ,specialty = cardiology ,
CLUSTER NUMBER 4
action = additem ,type = hr ,
CLUSTER NUMBER 5
action = additem ,type = hr ,
CLUSTER NUMBER 6
oauthor = oncnurse2 ,uagentfor = oncpat1 ,user = oncnurse2 ,
CLUSTER NUMBER 7
action = read ,uteam = carteam1 ,otreatingteam = carteam1 ,opatient = carpat1 ,oauthor = carnurse2 ,user = carpat1 ,
CLUSTER NUMBER 8
action = read ,uteam = carteam2 ,otreatingteam = carteam2 ,
CLUSTER NUMBER 9
action = addnote ,type = hr ,uteam = carteam2 ,opatient = oncpat1 ,
CLUSTER NUMBER 10
action = read ,oauthor = carnurse1 ,topic = note ,user = carnurse1 ,
CLUSTER NUMBER 11
action = addnote ,type = hr ,otreatingteam = oncteam1 ,oauthor = oncdoc1 ,topic = oncology ,user = carpat2 ,
CLUSTER NUMBER 12

In [49]:
datacorr = pd.DataFrame()
l=[
'oward'            ,
'uward'            ,
'uteam'            ,
'otreatingteam'    ,
'opatient'         ,
'oauthor'          ,
'topic'            ,
'specialty'        ,
'uagentfor'        ,
'user' ]

for i in range(k):
    print()
    print("CLUSTER NUMBER", i)
    clusTemp = ndf.loc[ndf['cluster'] == i]
    for col in df.columns:
        if col in l:
    #writer = csv.writer(f, dialect='excel')
            for col2 in df.columns:
                if col2 in l:
                    confusion_matrix = pd.crosstab(act[col], act[col2]).as_matrix()
                    datacorr = cramers_v(confusion_matrix)
                    #print('correlation between ',col,'and', col2,'is : ',datacorr)
                    temp=clusTemp[col].value_counts().to_dict()    
                    confusion_matrix = pd.crosstab(clusTemp[col], clusTemp[col2]).as_matrix()
                    temp = cramers_v(confusion_matrix)
                    #print (temp, datacorr)
                    if temp > datacorr :
                        #print (temp, datacorr)
                        c= temp - datacorr
                        if c >= 0.25:
                            #print (temp, datacorr,c)
                            print(col,'==', col2,',' ,end="")
                    else:
                        c= datacorr - temp 
                        if c >= 0.10:
                    
                            #print (temp, datacorr, c)
                            print(col,'!==', col2,',' ,end="")


CLUSTER NUMBER 0


/Users/aldairi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/aldairi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.



CLUSTER NUMBER 1
opatient == user ,user == opatient ,
CLUSTER NUMBER 2

CLUSTER NUMBER 3

CLUSTER NUMBER 4
oward == uward ,uward == oward ,
CLUSTER NUMBER 5

CLUSTER NUMBER 6
oauthor == user ,user == oauthor ,
CLUSTER NUMBER 7

CLUSTER NUMBER 8

CLUSTER NUMBER 9
opatient == uagentfor ,opatient == user ,uagentfor == opatient ,user == opatient ,
CLUSTER NUMBER 10

CLUSTER NUMBER 11
opatient == user ,user == opatient ,
CLUSTER NUMBER 12

CLUSTER NUMBER 13
uteam !== otreatingteam ,otreatingteam !== uteam ,opatient == user ,user == opatient ,

In [337]:
from sklearn.decomposition import PCA
def k_mean_distance(data, cx, cy, i_centroid, cluster_labels):
        distances = [np.sqrt((x-cx)**2+(y-cy)**2) for (x, y) in data[cluster_labels == i_centroid]]
        return distances
#clusters=km.fit_predict(data2D)
#centroids = km.cluster_centers_
#data2D=PCA(n_components=2).fit_transform(data)
distances = []
for i, (cx, cy) in enumerate(centroids):
    mean_distance = k_mean_distance(data2D, cx, cy, i, clusters)
    distances.append(mean_distance)

print(distances)

ValueError: could not convert string to float: 'true'

In [211]:
# Correlation in Each Cluster

with open("dcorr.csv",'w',newline='') as f:
    for col in df.columns:
        
        writer = csv.writer(f, dialect='excel')
        for col2 in df.columns:
            
            confusion_matrix = pd.crosstab(df[col], df[col2]).as_matrix()
            temp = cramers_v(confusion_matrix)
            writer.writerow({temp, col2 , col })
    f.close()



/Users/aldairi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':
